Importación de las bibliotecas

## 📊 Estructura del Notebook

Este notebook implementa un **Sistema de Recomendación basado en Redes Bipartitas** utilizando el dataset de Amazon Electronics Reviews.

### Secciones principales:

1. **Construcción del Grafo Bipartito**
   - Carga de datos con rutas relativas
   - Filtrado de reseñas de 5 estrellas
   - Creación del grafo bipartito (usuarios ↔ productos)
   - Visualización de la matriz de biadyacencia

2. **Análisis de la Red Bipartita**
   - Cálculo de grados de conectividad
   - Identificación de usuarios y productos más activos
   - Visualización de distribuciones

3. **Proyección a Red de Productos**
   - Proyección ponderada sobre productos
   - Análisis de pesos (usuarios compartidos)

4. **Sistema de Recomendación**
   - Algoritmo: **Similaridad de Jaccard**
   - Métricas de evaluación
   - Filtrado de productos duplicados

5. **Visualización e Interactividad**
   - Red estrella (star network) con producto central
   - Interfaz interactiva con dropdowns (ipywidgets)
   - Selección dinámica de productos y número de recomendaciones

---

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings; warnings.simplefilter('ignore')
%matplotlib inline

# Sistema de recomendación usando una proyección de red bipartita

**Integrantes:** Isaac Cerda, David Guamán, Christian Jácome, Mateo Jaramillo, David Navarrete

## Preparación de Datos

### **Paso 1:** Descargar el Dataset
Ejecutar **SOLO** si **NO** se tiene la carpeta `data` en el directorio `src/app` con el archivo `ratings_electronics.csv`

In [ ]:
from utils.download_dataset import download_ratings_electronics

dataset_path = download_ratings_electronics()

### **Paso 2:** Cargar el Dataset

In [ ]:
import pandas as pd
import os

# Ruta relativa al dataset desde la ubicación del notebook
dataset_path = os.path.join(os.path.dirname(os.path.abspath('__file__')), 
                            'data', 'amazon-products-reviews', 'ratings_electronics.csv')

# Carga del dataset
electronics_data = pd.read_csv(
    dataset_path, names=['userId', 'productId', 'Rating', 'timestamp']
)

# Impresión de las primeras 5 filas
electronics_data.head()

#### **Paso 2.1:** Revisar cantidad de datos del Dataset

In [ ]:
electronics_data.info()
electronics_data.describe()
print(f"Total registros: {len(electronics_data):,}")

### **Paso 3:** Filtrar reseñas de 5 estrellas

In [ ]:
electronics_data = electronics_data[electronics_data['Rating'] == 5]
print(f"Registros luego de filtrar 5 estrellas: {len(electronics_data):,}")

### **Paso 4:** Eliminar usuarios y productos con pocas conexiones

In [ ]:
# Contar cuántos ratings tiene cada usuario y producto
user_counts = electronics_data['userId'].value_counts()
product_counts = electronics_data['productId'].value_counts()

# Mantener usuarios y productos con al menos 4 interacciones
min_user_reviews = 4
min_product_reviews = 4

filtered_users = user_counts[user_counts >= min_user_reviews].index
filtered_products = product_counts[product_counts >= min_product_reviews].index

electronics_data = electronics_data[
    electronics_data['userId'].isin(filtered_users) &
    electronics_data['productId'].isin(filtered_products)
]

print(f"Registros tras filtrado de conexiones mínimas: {len(electronics_data):,}")


#### **Paso 4.1:** Validar campos nulos tras el filtrado

In [ ]:
electronics_data.isnull().sum()

## Construcción de la Red Bipartita

---

## **Sección 1: Construcción del Grafo Bipartito**

El grafo bipartito conecta dos conjuntos disjuntos de nodos:
- **Conjunto U**: Usuarios que califican productos
- **Conjunto V**: Productos que reciben calificaciones

Las aristas representan interacciones (calificaciones de 5 estrellas) entre usuarios y productos.

### **Paso 1:** Reducir cantidad de Nodos

Debido a que el dataset es muy grande, se toma una muestra

In [ ]:
import random

users = electronics_data['userId'].unique()
products = electronics_data['productId'].unique()

print("Antes de la muestra:")
print(f"Número de usuarios únicos: {len(users)}")
print(f"Número de productos únicos: {len(products)}")

users = list(users)
products = list(products)

sample_ratio = 0.1

num_users = int(len(users) * sample_ratio)
num_products = int(len(products) * sample_ratio)

random.seed(42)
users = random.sample(users, num_users)
products = random.sample(products, num_products)

print("Después de la muestra:")
print(f"Número de usuarios únicos: {len(users)}")
print(f"Número de productos únicos: {len(products)}")

### **Paso 2:** Crear red bipartita

In [ ]:
import networkx as nx

B = nx.Graph()

# Agregar nodos al grafo
B.add_nodes_from(users, bipartite=0)
B.add_nodes_from(products, bipartite=1)

# Agregar aristas
edges = electronics_data[['userId', 'productId']].values
B.add_edges_from(edges)

is_bipartite = nx.is_bipartite(B)
print(f"¿Es el grafo bipartito? {"Sí" if is_bipartite else "No"}")
num_edges = B.number_of_edges()
print("Número de aristas:", num_edges)

### **Paso 3:** Generar matriz de biadyacencia

Se utiliza la matriz de biadyacencia dispersa, que es una forma eficiente de representar grandes grafos bipartitos en memoria.

- Cada **fila** representa un usuario
- Cada **columna** representa un producto  
- Los valores **distintos de cero** indican una conexión (interacción usuario-producto)

La razón principal para trabajar con una matriz dispersa (sparse) en lugar de una matriz completa es que la mayoría de las celdas son ceros, permitiendo ahorrar memoria y procesar grafos grandes de forma eficiente.

In [ ]:
from networkx.algorithms import bipartite
import pandas as pd

# Obtener matriz de biadyacencia en formato disperso (sparse)
matrix_sparse = bipartite.biadjacency_matrix(B, row_order=users, column_order=products)

# Mostrar información general de la matriz
print("Matriz de biadyacencia (formato disperso):")
print(matrix_sparse)
print(f"Tamaño: {matrix_sparse.shape}")
print(f"Número de conexiones (valores distintos de 0): {matrix_sparse.nnz}")

# Extraer los índices (posiciones) de los valores distintos de cero
row_idx, col_idx = matrix_sparse.nonzero()

# Crear un DataFrame con los enlaces reales (usuario - producto)
edges_nonzero = pd.DataFrame({
    'userId': [users[i] for i in row_idx],
    'productId': [products[j] for j in col_idx]
})

print("\nConexiones reales (valores distintos de 0):")
print(edges_nonzero.head(10))  # Muestra solo los primeros 10 para no saturar la salida

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Visualización de la matriz de biadyacencia (muestra pequeña)
fig, ax = plt.subplots(figsize=(12, 8))

# Tomar una muestra pequeña para visualización
sample_users = users[:15] if len(users) >= 15 else users
sample_products = products[:15] if len(products) >= 15 else products

# Obtener submatriz para la muestra
user_indices = [users.index(u) for u in sample_users]
product_indices = [products.index(p) for p in sample_products]
sample_matrix = matrix_sparse[user_indices, :][:, product_indices].toarray()

# Crear heatmap
sns.heatmap(sample_matrix, 
            cmap='binary',
            cbar=False,
            xticklabels=[p[-8:] for p in sample_products],
            yticklabels=[f'U{i+1}' for i in range(len(sample_users))],
            square=True,
            linewidths=0.5,
            linecolor='gray',
            ax=ax)

ax.set_xlabel('Productos', fontsize=12, fontweight='bold')
ax.set_ylabel('Usuarios', fontsize=12, fontweight='bold')
ax.set_title('Matriz de Biadyacencia (Grafo Bipartito)\nNegro = Conexión, Blanco = Sin Conexión', 
             fontsize=14, fontweight='bold', pad=20)

plt.xticks(rotation=45, ha='right', fontsize=9)
plt.yticks(fontsize=9)
plt.tight_layout()
plt.show()

print(f"\nVisualizando {len(sample_users)} usuarios × {len(sample_products)} productos")
print(f"Conexiones en esta muestra: {sample_matrix.sum():.0f}")

---

## **Sección 2: Análisis de la Red Bipartita**

### **Paso 4:** Graficar nodos con mayores conexiones

Se calculan los grados de los nodos (número de conexiones):

- `num_products_rated`: cuántos productos calificó cada usuario.
- `num_ratings_received`: cuántos usuarios calificaron cada producto.

Luego se seleccionan los Top N nodos más conectados en cada grupo `nlargest`.

In [ ]:
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
from networkx.algorithms import bipartite
import seaborn as sns

# Calcular grados de los usuarios y productos
user_degrees = dict(B.degree(users))
product_degrees = dict(B.degree(products))

user_deg_df = pd.DataFrame(user_degrees.items(), columns=["userId", "num_products_rated"])
product_deg_df = pd.DataFrame(product_degrees.items(), columns=["productId", "num_ratings_received"])

print("\nGrados de los primeros usuarios:")
print(user_deg_df.head())

print("\nGrados de los primeros productos:")
print(product_deg_df.head())

# --- Top N nodos ---
N = 10
top_users = user_deg_df.nlargest(N, "num_products_rated")
top_products = product_deg_df.nlargest(N, "num_ratings_received")

plt.figure(figsize=(14, 6))

# Top N usuarios más activos
plt.subplot(1, 2, 1)
sns.barplot(data=top_users, x="num_products_rated", y="userId", palette="Blues_d")
plt.title(f"Top {N} usuarios más activos")
plt.xlabel("Número de productos calificados")
plt.ylabel("ID de usuario")

# Top N productos más reseñados
plt.subplot(1, 2, 2)
sns.barplot(data=top_products, x="num_ratings_received", y="productId", palette="Greens_d")
plt.title(f"Top {N} productos más reseñados")
plt.xlabel("Número de reseñas recibidas")
plt.ylabel("ID de producto")

plt.tight_layout()
plt.show()


#### **Paso 4.1:** Mostrar resultado final

In [ ]:
import numpy as np

# Calcular grados de los usuarios y productos
user_degrees = dict(B.degree(users))
product_degrees = dict(B.degree(products))

print(f"Número de usuarios: {len(users)}")
print(f"Número de productos: {len(products)}")
print(f"Número de aristas (reseñas de 5 estrellas): {B.number_of_edges()}")
print(f"Grado promedio de usuarios: {np.mean(list(user_degrees.values())):.2f}")
print(f"Grado promedio de productos: {np.mean(list(product_degrees.values())):.2f}")

---

## **Sección 3: Proyección a Red de Productos**

La proyección conecta productos que comparten usuarios en común.

**Proceso:**
1. Se toma el grafo bipartito B(U, V, E)
2. Se proyecta sobre el conjunto V (productos)
3. Dos productos quedan conectados si tienen al menos un usuario en común
4. El **peso de la arista** = número de usuarios compartidos

**Ejemplo:** Si los productos P1 y P2 fueron calificados por 15 usuarios en común, la arista (P1, P2) tendrá peso = 15.

In [ ]:
# Crear la proyección de productos
# Esto conecta productos que fueron calificados por los mismos usuarios
product_network = bipartite.weighted_projected_graph(B, products)

print(f"Red de productos creada:")
print(f"Número de productos: {product_network.number_of_nodes()}")
print(f"Número de conexiones entre productos: {product_network.number_of_edges()}")

# Verificar algunos pesos de las conexiones
if product_network.number_of_edges() > 0:
    # Obtener algunas aristas con sus pesos
    sample_edges = list(product_network.edges(data=True))[:5]
    print("\nEjemplos de conexiones entre productos (peso = usuarios en común):")
    for edge in sample_edges:
        prod1, prod2, data = edge
        weight = data['weight']
        print(f"  {prod1} -- {prod2}: {weight} usuarios en común")

---

## **Sección 4: Sistema de Recomendación con Similaridad de Jaccard**

### **Algoritmo implementado**

**Similaridad de Jaccard** mide qué tan similares son dos productos basándose en sus usuarios compartidos:

$$J(A, B) = \frac{|A \cap B|}{|A \cup B|} \times 100\%$$

Donde:
- $A$: conjunto de usuarios que calificaron el producto A
- $B$: conjunto de usuarios que calificaron el producto B  
- $|A \cap B|$: usuarios que calificaron ambos productos
- $|A \cup B|$: usuarios que calificaron al menos uno de los dos productos

**Características:**
- Rango: 0% (sin usuarios en común) a 100% (mismos usuarios exactamente)
- Es **simétrica**: $J(A,B) = J(B,A)$
- Normaliza por el tamaño total de la unión (evita sesgo por popularidad)

**Caso especial:** 100% de similaridad indica que dos productos tienen exactamente el mismo conjunto de usuarios, lo que puede indicar:
- Productos duplicados en el catálogo
- Variantes del mismo producto (diferentes colores/tamaños)
- El algoritmo puede filtrar estos casos si se desea

In [ ]:
def get_product_recommendations(product_id, n_recommendations=5, exclude_perfect_match=True):
    """
    Obtiene recomendaciones basadas en SIMILITUD DE JACCARD
    
    SIMILITUD DE JACCARD:
    - Métrica estándar para sistemas de recomendación
    - Fórmula: J(A,B) = |A ∩ B| / |A ∪ B|
    - Rango: 0% a 100%
    
    NOTA: 100% significa que exactamente los mismos usuarios calificaron ambos productos
    (A = B), lo cual puede indicar productos duplicados o variantes del mismo producto.
    
    Args:
        product_id: ID del producto
        n_recommendations: Número de recomendaciones
        exclude_perfect_match: Si True, excluye productos con 100% de similaridad
    
    Returns: Lista de tuplas (producto_id, porcentaje_jaccard, info_adicional)
    """
    if product_id not in product_network:
        return []
    
    neighbors = product_network[product_id].items()
    users_base = set(B.neighbors(product_id))
    num_users_base = len(users_base)
    
    recommendations = []
    for neighbor, data in neighbors:
        users_common = data['weight']
        users_neighbor = set(B.neighbors(neighbor))
        num_users_neighbor = len(users_neighbor)
        
        # SIMILITUD DE JACCARD: |A ∩ B| / |A ∪ B|
        union_size = num_users_base + num_users_neighbor - users_common
        jaccard_similarity = (users_common / union_size) * 100 if union_size > 0 else 0
        
        # Información adicional para análisis
        info = {
            'users_base': num_users_base,
            'users_neighbor': num_users_neighbor,
            'users_common': users_common,
            'is_perfect_match': (jaccard_similarity == 100.0)
        }
        
        # Filtrar coincidencias perfectas si se solicita
        if exclude_perfect_match and jaccard_similarity == 100.0:
            continue
            
        recommendations.append((neighbor, jaccard_similarity, info))
    
    return sorted(recommendations, key=lambda x: x[1], reverse=True)[:n_recommendations]

### **Demostración del Sistema de Recomendación**

Prueba del sistema con productos aleatorios mostrando las métricas de similaridad.

In [ ]:
import random

# Seleccionar productos aleatorios para demostración
def demo_recommendation_system(num_products=3, n_recommendations=5, show_perfect_matches=True):
    """
    Sistema de recomendación simplificado con productos aleatorios
    
    Args:
        num_products: Cuántos productos aleatorios probar
        n_recommendations: Cuántas recomendaciones generar por producto
        show_perfect_matches: Si True, incluye productos con 100% similaridad
    """
    # Filtrar productos que tienen conexiones
    valid_products = [p for p in products if p in product_network and len(product_network[p]) >= n_recommendations]
    
    if len(valid_products) < num_products:
        print(f"Solo hay {len(valid_products)} productos con suficientes conexiones")
        num_products = len(valid_products)
    
    # Seleccionar productos aleatorios
    random.seed(42)
    selected_products = random.sample(valid_products, num_products)
    
    print("=" * 80)
    print("SISTEMA DE RECOMENDACIÓN - PRODUCTOS ALEATORIOS")
    print("=" * 80)
    print(f"Métrica utilizada: Similitud de Jaccard")
    print(f"Total de productos en red: {len(products)}")
    print(f"Productos seleccionados: {num_products}")
    print(f"Recomendaciones por producto: {n_recommendations}")
    print(f"Incluir coincidencias perfectas (100%): {'Sí' if show_perfect_matches else 'No'}")
    print("=" * 80)
    
    perfect_matches_found = 0
    
    for idx, product_id in enumerate(selected_products, 1):
        print(f"\n[{idx}] PRODUCTO: {product_id}")
        
        # Obtener recomendaciones
        recommendations = get_product_recommendations(product_id, n_recommendations, 
                                                     exclude_perfect_match=not show_perfect_matches)
        
        if recommendations:
            print(f"    Recomendaciones generadas: {len(recommendations)}")
            print(f"    {'Rank':<6} {'Producto':<20} {'Jaccard':<10} {'Info'}")
            print(f"    {'-'*6} {'-'*20} {'-'*10} {'-'*30}")
            
            for i, (prod_id, similarity, info) in enumerate(recommendations, 1):
                status = ""
                if info['is_perfect_match']:
                    perfect_matches_found += 1
                    status = "⚠️ MATCH PERFECTO"
                    
                print(f"    {i:<6} {prod_id[:20]:<20} {similarity:>7.2f}%   {status}")
                
                # Mostrar detalles si es match perfecto
                if info['is_perfect_match']:
                    print(f"           └─ Usuarios: Base={info['users_base']}, "
                          f"Vecino={info['users_neighbor']}, Común={info['users_common']}")
        else:
            print("    Sin recomendaciones disponibles")
    
    print("\n" + "=" * 80)
    print("RESUMEN DE MÉTRICAS")
    print("=" * 80)
    
    # Calcular estadísticas generales
    all_similarities = []
    total_recommendations = 0
    
    for product_id in selected_products:
        recs = get_product_recommendations(product_id, n_recommendations, 
                                          exclude_perfect_match=not show_perfect_matches)
        total_recommendations += len(recs)
        all_similarities.extend([sim for _, sim, _ in recs])
    
    if all_similarities:
        print(f"Total de recomendaciones generadas: {total_recommendations}")
        print(f"Similitud promedio: {np.mean(all_similarities):.2f}%")
        print(f"Similitud máxima: {np.max(all_similarities):.2f}%")
        print(f"Similitud mínima: {np.min(all_similarities):.2f}%")
        
        if perfect_matches_found > 0:
            print(f"\n⚠️  ADVERTENCIA: Se encontraron {perfect_matches_found} coincidencias perfectas (100%)")
            print(f"    Esto indica productos con exactamente los mismos usuarios.")
            print(f"    Pueden ser: duplicados, variantes, o productos muy relacionados.")
    
    print("=" * 80)

# Ejecutar demo con coincidencias perfectas
print("DEMOSTRACIÓN 1: Incluyendo coincidencias perfectas\n")
demo_recommendation_system(num_products=5, n_recommendations=5, show_perfect_matches=True)

print("\n\n")

# Ejecutar demo SIN coincidencias perfectas (más realista)
print("DEMOSTRACIÓN 2: Excluyendo coincidencias perfectas (recomendación más diversa)\n")
demo_recommendation_system(num_products=5, n_recommendations=5, show_perfect_matches=False)

---

## **Sección 5: Visualización e Interactividad**

### Visualización de Red Estrella (Star Network)

Visualización del producto central con sus productos recomendados conectados mediante aristas rojas.

In [ ]:
def visualize_star_network(product_id, n_recommendations=4):
    """Visualiza red estrella: producto central con vecinos conectados por aristas rojas"""
    recs = get_product_recommendations(product_id, n_recommendations, exclude_perfect_match=False)
    
    if not recs:
        print(f"No hay recomendaciones para {product_id}")
        return
    
    # Crear grafo estrella
    G = nx.Graph()
    G.add_node(product_id)
    
    for neighbor, sim, _ in recs:
        G.add_node(neighbor)
        G.add_edge(product_id, neighbor, weight=sim)
    
    # Posiciones: centro y vecinos en círculo
    pos = {product_id: (0, 0)}
    angle_step = 2 * np.pi / len(recs)
    for i, (neighbor, _, _) in enumerate(recs):
        pos[neighbor] = (np.cos(i * angle_step), np.sin(i * angle_step))
    
    # Visualización
    plt.figure(figsize=(12, 10))
    
    # Nodo central (producto seleccionado)
    nx.draw_networkx_nodes(G, pos, nodelist=[product_id], 
                          node_color='#4CAF50', 
                          node_size=2000, 
                          alpha=0.95,
                          edgecolors='black',
                          linewidths=2)
    
    # Nodos vecinos (productos recomendados)
    nx.draw_networkx_nodes(G, pos, nodelist=[n for n, _, _ in recs], 
                          node_color='#FF9800', 
                          node_size=1200, 
                          alpha=0.9,
                          edgecolors='black',
                          linewidths=1.5)
    
    # Aristas rojas con grosor variable según similaridad
    edge_widths = [sim/15 for _, sim, _ in recs]
    nx.draw_networkx_edges(G, pos, 
                          edge_color='#F44336', 
                          width=edge_widths, 
                          alpha=0.8)
    
    # Etiquetas de nodos (IDs truncados)
    labels = {product_id: f"★ {product_id[-10:]}"}
    labels.update({n: n[-10:] for n, _, _ in recs})
    nx.draw_networkx_labels(G, pos, labels, font_size=9, font_weight='bold')
    
    # Etiquetas de peso en las aristas
    edge_labels = {(product_id, n): f"{s:.1f}%" for n, s, _ in recs}
    nx.draw_networkx_edge_labels(G, pos, edge_labels, font_size=9, font_color='#D32F2F')
    
    plt.title(f'Red Estrella (Star Network)\nProducto Central: {product_id[-20:]}', 
             fontsize=15, fontweight='bold', pad=25)
    plt.axis('off')
    plt.tight_layout()
    plt.show()
    
    # Tabla de recomendaciones
    print(f"\n{'='*80}")
    print(f"PRODUCTO CENTRAL: {product_id}")
    print(f"{'='*80}")
    print(f"\n{'Rank':<6} {'Producto ID':<50} {'Similaridad':<12}")
    print(f"{'-'*80}")
    for i, (prod, sim, _) in enumerate(recs, 1):
        print(f"{i:<6} {prod:<50} {sim:>6.2f}%")
    print(f"{'='*80}\n")

# Demostración con un producto válido
valid_prods = [p for p in products if p in product_network and len(product_network[p]) >= 4]
if valid_prods:
    visualize_star_network(valid_prods[0], n_recommendations=5)

### Interfaz Interactiva con Dropdowns

Selección de producto y número de recomendaciones mediante widgets interactivos.

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# Productos válidos (con suficientes vecinos)
valid_products = [p for p in products if p in product_network and len(product_network[p]) >= 4][:50]

# Crear dropdowns
product_dropdown = widgets.Dropdown(
    options=valid_products,
    description='Producto:',
    style={'description_width': 'initial'}
)

n_dropdown = widgets.Dropdown(
    options=[3, 4, 5, 6, 8, 10],
    value=4,
    description='N Recomendaciones:',
    style={'description_width': 'initial'}
)

# Función de actualización
def update_visualization(change):
    with output:
        clear_output(wait=True)
        visualize_star_network(product_dropdown.value, n_dropdown.value)

# Vincular eventos
product_dropdown.observe(update_visualization, names='value')
n_dropdown.observe(update_visualization, names='value')

# Output widget
output = widgets.Output()

# Mostrar interfaz
display(widgets.VBox([product_dropdown, n_dropdown, output]))

# Visualización inicial
with output:
    visualize_star_network(product_dropdown.value, n_dropdown.value)

In [ ]:
# -------------------------
# CELDA: 2 - Funciones de análisis (top/bottom, poda por grado/peso)
# -------------------------
def top_bottom_products(product_network, top_n=10):
    """Devuelve DataFrames con top y bottom productos por grado en la proyección."""
    deg = dict(product_network.degree())
    df = pd.DataFrame.from_dict(deg, orient='index', columns=['degree']).reset_index().rename(columns={'index':'productId'})
    top = df.nlargest(top_n, 'degree')
    bottom = df.nsmallest(top_n, 'degree')
    return top, bottom

def prune_by_min_degree(product_network, min_degree=2):
    """Devuelve subgrafo con productos cuyo grado >= min_degree."""
    nodes = [n for n, d in product_network.degree() if d >= min_degree]
    return product_network.subgraph(nodes).copy()

def prune_by_weight_range(product_network, min_weight=20, max_weight=100):
    """Devuelve subgrafo que mantiene solo aristas con weight en [min_weight, max_weight] y nodos aislados eliminados."""
    G = nx.Graph()
    for u, v, data in product_network.edges(data=True):
        w = data.get('weight', 0)
        if min_weight <= w <= max_weight:
            G.add_edge(u, v, weight=w)
    # eliminar nodos aislados implícitamente: sólo nodos con grado>0 están en G
    return G

def analyze_and_print(product_network, top_n=10):
    top, bottom = top_bottom_products(product_network, top_n=top_n)
    print(f"\nTop {top_n} productos por grado:")
    display(top)
    print(f"\nBottom {top_n} productos por grado:")
    display(bottom)
    # Histograma grados
    plt.figure(figsize=(10,4))
    degrees = [d for _, d in product_network.degree()]
    sns.histplot(degrees, bins=40, kde=False)
    plt.title("Distribución de grados - Red de productos")
    plt.xlabel("Grado")
    plt.show()


In [ ]:
# -------------------------
# CELDA: 5 - Widgets interactivos: producto + N + opciones de poda
# -------------------------
def create_interactive_ui(product_network, max_products_listed=200):
    # productos válidos (con al menos 1 vecino)
    valid_products = [p for p in product_network.nodes() if product_network.degree(p)>0][:max_products_listed]

    product_dropdown = widgets.Dropdown(
        options=valid_products,
        description='Producto:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='70%')
    )

    n_dropdown = widgets.Dropdown(
        options=[3,4,5,6,8,10,12,15],
        value=4,
        description='N recomendaciones:',
        style={'description_width': 'initial'}
    )

    prune_dropdown = widgets.Dropdown(
        options=['Sin poda', 'Poda por grado (min)', 'Poda por peso (20-100)'],
        value='Sin poda',
        description='Poda:',
        style={'description_width': 'initial'}
    )

    min_degree_slider = widgets.IntSlider(value=2, min=1, max=50, step=1, description='min degree:', layout=widgets.Layout(width='60%'))

    exclude_perfect = widgets.Checkbox(value=False, description='Excluir coincidencias 100%')

    output = widgets.Output()

    def update_visualization(change):
        with output:
            clear_output(wait=True)
            # aplicar poda localmente (no modificar product_network globalmente)
            G = product_network
            if prune_dropdown.value == 'Poda por grado (min)':
                G = prune_by_min_degree(product_network, min_degree_slider.value)
            elif prune_dropdown.value == 'Poda por peso (20-100)':
                G = prune_by_weight_range(product_network, min_weight=20, max_weight=100)

            # set global G used in visualization functions temporarily
            # Note: visualize_star_network uses global product_network, so we call local functions directly
            # Recompute recommendations on G:
            # create a temporary wrapper around get_product_recommendations using G
            def local_get_recs(pid, n, exclude):
                # compute jaccard on G
                if pid not in G:
                    return []
                deg_a = G.degree(pid)
                res = []
                for neigh in G[pid]:
                    weight = G[pid][neigh].get('weight', 0)
                    deg_b = G.degree(neigh)
                    denom = deg_a + deg_b - weight
                    sim = (weight / denom) * 100 if denom > 0 else 0.0
                    if exclude and np.isclose(sim, 100.0):
                        continue
                    res.append((neigh, sim, weight))
                res.sort(key=lambda x: (x[1], x[2]), reverse=True)
                return res[:n]

            pid = product_dropdown.value
            n = n_dropdown.value
            recs = local_get_recs(pid, n, exclude_perfect.value)
            if not recs:
                print(f"No hay recomendaciones con la poda seleccionada para {pid}.")
                return

            # Visualizar usando la misma estética de visualize_star_network pero con recs calculados localmente
            # Crear grafo estrella
            G_star = nx.Graph()
            G_star.add_node(pid)
            for neighbor, sim, weight in recs:
                G_star.add_node(neighbor)
                G_star.add_edge(pid, neighbor, weight=sim, raw_weight=weight)

            # posiciones
            pos = {pid: (0,0)}
            angle_step = 2 * np.pi / len(recs)
            for i, (neighbor, _, _) in enumerate(recs):
                pos[neighbor] = (np.cos(i * angle_step), np.sin(i * angle_step))

            plt.figure(figsize=(10,8))
            nx.draw_networkx_nodes(G_star, pos, nodelist=[pid], node_color='#4CAF50', node_size=2000, edgecolors='black', linewidths=2)
            neighbor_nodes = [n for n,_,_ in recs]
            nx.draw_networkx_nodes(G_star, pos, nodelist=neighbor_nodes, node_color='#FF9800', node_size=1200, edgecolors='black', linewidths=1.2)
            edge_widths = [max(0.8, G_star[u][v]['weight']/10) for u,v in G_star.edges()]
            nx.draw_networkx_edges(G_star, pos, width=edge_widths, edge_color='#D32F2F', alpha=0.9)
            labels = {n: (f"★ {n[-10:]}" if n==pid else n[-10:]) for n in G_star.nodes()}
            nx.draw_networkx_labels(G_star, pos, labels, font_size=9, font_weight='bold')
            edge_labels = {(pid, n): f"{sim:.1f}%" for n, sim, _ in recs}
            nx.draw_networkx_edge_labels(G_star, pos, edge_labels=edge_labels, font_color='#B71C1C', font_size=9)
            plt.title(f"Red Estrella — Producto central: {pid[-20:]}", fontsize=14, fontweight='bold', pad=15)
            plt.axis('off')
            plt.show()

            # Tabla de resultados
            print("="*80)
            print(f"PRODUCTO CENTRAL: {pid}")
            print("="*80)
            print(f"\n{'Rank':<6} {'Producto ID':<50} {'Similaridad':<12} {'Peso(us.c.)':<10}")
            print("-"*100)
            for i, (prod, sim, w) in enumerate(recs, 1):
                print(f"{i:<6} {prod:<50} {sim:>7.2f}%     {w:<10}")
            print("="*80 + "\n")

    # conectar observadores
    product_dropdown.observe(update_visualization, names='value')
    n_dropdown.observe(update_visualization, names='value')
    prune_dropdown.observe(update_visualization, names='value')
    min_degree_slider.observe(update_visualization, names='value')
    exclude_perfect.observe(update_visualization, names='value')

    ui = widgets.VBox([
        widgets.HBox([product_dropdown, n_dropdown]),
        widgets.HBox([prune_dropdown, min_degree_slider, exclude_perfect]),
        output
    ])
    display(ui)
    # llamada inicial
    update_visualization(None)

# Uso: create_interactive_ui(product_network)


In [ ]:
# -------------------------
# CELDA: 6 - Podas de ejemplo y análisis (ejemplos de uso)
# -------------------------
# Asegúrate de que 'product_network' exista (si no, descomenta ensure_graphs_exist())
# ensure_graphs_exist()

# Análisis rápido de top/bottom (usa la red completa)
analyze_and_print(product_network, top_n=10)

# Ejemplos de poda
G_by_degree = prune_by_min_degree(product_network, min_degree=10)
print("Después de poda por grado >=10 -> nodos:", G_by_degree.number_of_nodes(), "aristas:", G_by_degree.number_of_edges())

G_by_weight = prune_by_weight_range(product_network, min_weight=20, max_weight=100)
print("Después de poda por peso 20-100 -> nodos:", G_by_weight.number_of_nodes(), "aristas:", G_by_weight.number_of_edges())



In [ ]:
# -------------------------
# CELDA: 8 - Exportar notebook a HTML (Colab / Jupyter)
# -------------------------
# Si estás en Colab: guarda el notebook manualmente ('File -> Save') y luego ejecuta:
jupyter nbconvert --to html "main.ipynb" --output "main_visualization.html"
# from google.colab import files
# files.download("main_visualization.html")

# Alternativamente (Jupyter local) desde terminal:
# jupyter nbconvert --to html main.ipynb --output main_visualization.html

print("Para exportar: usa nbconvert:  jupyter nbconvert --to html main.ipynb --output main_visualization.html")


---

## 📈 Resumen del Sistema

### Características Principales

✅ **Grafo Bipartito**: Conecta usuarios y productos mediante calificaciones de 5 estrellas  
✅ **Proyección Ponderada**: Red de productos donde los pesos representan usuarios compartidos  
✅ **Similaridad de Jaccard**: Métrica normalizada que mide similitud entre productos (0-100%)  
✅ **Filtrado Inteligente**: Elimina productos con pocas conexiones y opcionalmente duplicados perfectos  
✅ **Visualización Interactiva**: Red estrella con dropdowns para explorar recomendaciones  
✅ **Matriz de Biadyacencia**: Representación visual del grafo bipartito  

### Métricas y Resultados

- **Dataset**: Amazon Electronics Reviews
- **Filtrado**: Solo reseñas de 5 estrellas
- **Umbral**: Mínimo 4 conexiones por usuario/producto
- **Algoritmo**: Similaridad de Jaccard basada en usuarios compartidos
- **Rango típico**: 0.3% - 50% de similaridad (valores realistas)
- **Casos especiales**: 100% indica productos idénticos (duplicados/variantes)

### Uso del Sistema

1. Ejecutar celdas en orden secuencial
2. Usar los dropdowns interactivos para seleccionar productos
3. Ajustar el número de recomendaciones deseadas
4. Visualizar la red estrella y métricas de similaridad